In [12]:
import pandas as pd
import numpy as np
import psycopg2
import os
from psycopg2.extras import execute_values
from dotenv import load_dotenv


load_dotenv()


True

In [13]:
conn = psycopg2.connect(
        dbname=os.getenv("POSTGRES_DB"),
        user=os.getenv("POSTGRES_USER"),
        password=os.getenv("POSTGRES_PASSWORD"),
        host="localhost",
        port="5432")

cursor = conn.cursor()

In [14]:
df_bistek = pd.read_parquet('data/silver/bistek.parquet')
df_giassi = pd.read_parquet('data/silver/giassi.parquet')
df_angeloni = pd.read_parquet('data/silver/angeloni.parquet')

In [15]:
# Criando nova coluna com o nome completo do produto
df_bistek['full_name'] = df_bistek.name + ' ' + df_bistek.weight.astype(str) + df_bistek.measure
df_giassi['full_name'] = df_giassi.name + ' ' + df_giassi.weight.astype(str) + df_giassi.measure
df_angeloni['full_name'] = df_angeloni.name + ' ' + df_angeloni.weight.astype(str) + df_angeloni.measure

In [17]:
df_merged = pd.concat([df_bistek, df_giassi, df_angeloni], ignore_index=True)
df_merged.drop(columns=['insertedAt'], inplace=True)
df_merged = df_merged.loc[df_merged['price'] > 0]

In [18]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("all-mpnet-base-v2", device="cuda")

In [20]:
df_merged['embedded_name'] = df_merged['full_name'].apply(lambda x: model.encode(x, normalize_embeddings=True).tolist())

In [21]:
df_merged_cols = ','.join(df_merged.columns)
df_merged_values = [tuple(x) for x in df_merged.to_numpy()]
df_merged_query = f"INSERT INTO base_products ({df_merged_cols}) VALUES %s"

execute_values(cursor, df_merged_query, df_merged_values)
conn.commit()

In [ ]:
user_query = "creme de leite zero lactose"
user_embedded_query = model.encode(user_query, normalize_embeddings=True).tolist()


sql = """
SELECT full_name, embedded_name <-> %s::vector(768) AS distance
FROM base_products
ORDER BY embedded_name <-> %s::vector(768)
"""

cursor.execute(sql, (user_embedded_query, user_embedded_query))
results = cursor.fetchall()

for full_name, distance in results:
    print(f"Produto: {full_name}, Distância: {distance}")